**Importing the necessary dependencies**

In [1]:
!pip install -q transformers accelerate torch
!pip install -q arize-phoenix gradio pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322

In [2]:
import gradio as gr
import phoenix as px
import pandas as pd
import torch
from transformers import pipeline
import matplotlib.pyplot as plt
from datetime import datetime
import uuid
from huggingface_hub import login
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

**Using the HuggingFace API to import the LLM**

In [3]:
login()

In [4]:
model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [5]:
import phoenix as px
from phoenix.trace import TraceDataset
import time

In [6]:
#px.launch_app(run_in_thread=True)

In [7]:
trace_data = []
chat_history = []

In [8]:
df = pd.DataFrame(columns=["prompt", "response", "rating", "user_feedback"])
df["rating"] = None
df["user_feedback"] = None

In [36]:
def chatbot_response(user_input, history):
    start_time = time.time()
    trace_id = str(uuid.uuid4())  # Unique trace ID
    span_id = str(uuid.uuid4())   # Unique span ID

    # Generate response
    inputs = tokenizer(user_input, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_length=200, do_sample = True, temprature = 0.7, top_p = 0.7, repetition_penalty= 1.25)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    response_text = response_text.replace("\n\n", " ").replace("\n", " ")

    # Append message in OpenAI format
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": response_text})

    end_time = time.time()

    # Log the trace with Phoenix-compatible fields
    trace_entry = {
        "context.trace_id": trace_id,
        "context.span_id": span_id,
        "name": "chatbot_response",
        "span_kind": "LLM",
        "start_time": start_time,
        "end_time": end_time,
        "parent_id": None,
        "status_code": "OK",
        "status_message": "Success",
        "prompt": user_input,
        "response": response_text,
        "rating": None,
        "user_feedback": None
    }
    trace_data.append(trace_entry)

    return history

In [28]:
# def chatbot_response(user_input, history):
#     start_time = time.time()
#     trace_id = str(uuid.uuid4())  # Unique trace ID
#     span_id = str(uuid.uuid4())   # Unique span ID

#     # Generate response
#     inputs = tokenizer(user_input, return_tensors="pt").to("cuda")
#     outputs = model.generate(
#         **inputs,
#         max_length=500,
#         do_sample=True,
#         temperature=0.3,
#         top_p=0.7
#     )
#     response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     response_text = response_text.replace("\n\n", " ").replace("\n", " ")

#     # Append message to chat history
#     history.append(("User", user_input))
#     history.append(("Chatbot", response_text))

#     end_time = time.time()

#     # Log the trace with Phoenix-compatible fields
#     trace_entry = {
#         "context.trace_id": trace_id,
#         "context.span_id": span_id,
#         "name": "chatbot_response",
#         "span_kind": "LLM",
#         "start_time": start_time,
#         "end_time": end_time,
#         "parent_id": None,
#         "status_code": "OK",
#         "status_message": "Success",
#         "prompt": user_input,
#         "response": response_text,
#         "rating": None,
#         "user_feedback": None
#     }
#     trace_data.append(trace_entry)

#     return history

In [29]:
# if not trace_data:
#     trace_data.append({
#         "context.trace_id": str(uuid.uuid4()),
#         "context.span_id": str(uuid.uuid4()),
#         "parent_id": None,
#         "start_time": datetime.utcnow().isoformat(),
#         "end_time": datetime.utcnow().isoformat(),
#         "status_code": "OK",
#         "status_message": "No data yet",
#         "span_kind": "LLM",
#         "name": "Placeholder",
#         "prompt": "",
#         "response": "",
#         "rating": None,
#         "user_feedback": None
#     })

In [37]:
def export_data():
    trace_df = pd.DataFrame(trace_data)
    trace_df.to_csv("trace_data.csv", index=False)
    trace_df.to_json("trace_data.json", orient="records")

In [38]:
def save_feedback(index, rating, feedback_text):
    if index < len(trace_data):
        trace_data[index]["rating"] = rating
        trace_data[index]["user_feedback"] = feedback_text
        export_data()  # Save after feedback update
        return "✅ Feedback saved successfully!"
    return "⚠️ Invalid selection!"

In [39]:
def plot_rating_distribution():
    df = pd.DataFrame(trace_data)
    if df.empty or "rating" not in df.columns or df["rating"].dropna().empty:
        return None  # No ratings yet

    plt.figure(figsize=(5,3))
    df["rating"].dropna().astype(int).value_counts().sort_index().plot(kind="bar", color="skyblue")
    plt.xlabel("Rating (1-5)")
    plt.ylabel("Count")
    plt.title("Chatbot Rating Distribution")
    plt.xticks(rotation=0)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.savefig("rating_chart.png")
    return "rating_chart.png"

In [40]:
def summary_stats():
    df = pd.DataFrame(trace_data)
    if df.empty:
        return "📊 No feedback available yet."

    total_feedbacks = df["rating"].count()
    good_count = df[df["rating"] >= 3]["rating"].count()
    bad_count = total_feedbacks - good_count
    return f"📊 Total Feedbacks: {total_feedbacks}\n👍 Good Responses: {good_count}\n👎 Bad Responses: {bad_count}"

## **UI OF THE CHATBOT**

In [41]:
with gr.Blocks() as chatbot_ui:
    gr.Markdown("# 🤖 AI Chatbot with Phoenix Integration")

    with gr.Tab("Chatbot"):
        chat_history_ui = gr.Chatbot(type="messages")
        user_input = gr.Textbox(label="Type your message here...", interactive=True)
        send_button = gr.Button("Send")

    with gr.Tab("Feedback"):
        gr.Markdown("### 📝 Rate Chatbot Responses")
        index_slider = gr.Slider(0, len(trace_data)-1, step=1, label="Select Response")
        rating_slider = gr.Slider(1, 5, step=1, label="Rate (1 = Poor, 5 = Excellent)")
        feedback_textbox = gr.Textbox(label="Feedback", placeholder="Write your feedback here...")
        save_button = gr.Button("💾 Save Feedback")
        feedback_status = gr.Text()

    with gr.Tab("Analytics"):
        gr.Markdown("### 📊 Feedback Analytics")
        analytics_display = gr.Textbox(summary_stats(), interactive=False)
        rating_chart = gr.Image(value=plot_rating_distribution() or None)

    # Bind chatbot interactions
    send_button.click(fn=chatbot_response, inputs=[user_input, chat_history_ui], outputs=chat_history_ui)

    # Bind feedback submission
    save_button.click(save_feedback, inputs=[index_slider, rating_slider, feedback_textbox], outputs=feedback_status)

In [42]:
chatbot_ui.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7063a389aba87147ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [24]:
export_data()
trace_df = pd.DataFrame(trace_data)
trace_dataset = px.TraceDataset(dataframe=trace_df)
px.launch_app(trace=trace_dataset)

🌍 To view the Phoenix app in your browser, visit https://prpsfkq8mt4-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
